In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import torch
from datasets import load_dataset

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
import os
from dataloader.dataloader import NMTDataset
from torch.utils.data import DataLoader, random_split
from models.transformer import Transformer
from torchsummary import torchsummary
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchmetrics import ExactMatch, Accuracy


device = torch.device("cuda")
cpu_device = torch.device("cpu")

/data3/home/karmpatel/miniconda3/envs/torch310/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_train = 40000

raw_dataset = load_dataset('cfilt/iitb-english-hindi', split=f"train[:{n_train}]")
print(len(raw_dataset))

# determine max length
val_ds_size = 4000
d_model = 512
drop_prob = 0.1
n_h = 8
d_ff = 2048
d_k = d_model/n_h
n_layers = 6
seq_len = 130
B = 256
lr = 1e-4
n_epochs = 10

train_ds_size = n_train - val_ds_size
train_dataset, val_dataset = random_split(raw_dataset, [train_ds_size, val_ds_size], generator=torch.Generator().manual_seed(42))


# val_dataset = load_dataset('cfilt/iitb-english-hindi', split="validation")
print(len(val_dataset))

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset))
vl = next(iter(val_dataloader))


for lang in ["hi", "en"]:
    tokenizer_path = f"dataset/vocab_{lang}_{n_train}.json"
    if os.path.exists(tokenizer_path):
        # load tokenizer
        tokenizer = Tokenizer.from_file(tokenizer_path)
    else:
        tokenizer = Tokenizer(WordLevel(unk_token="<unk>"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=['<unk>', '<pad>', '<sos>', '<eos>'], min_frequency=2)
        tokenizer.train_from_iterator(tr['translation'][lang] + vl['translation'][lang]+ tt['translation'][lang], trainer=trainer)

        # save tokenizer
        tokenizer.save(tokenizer_path)


tokenizer_src = Tokenizer.from_file(f"dataset/vocab_en_{n_train}.json")
tokenizer_tar = Tokenizer.from_file(f"dataset/vocab_hi_{n_train}.json")

val_ds = NMTDataset(tokenizer_src, tokenizer_tar, val_dataloader, max_length=seq_len)
print(len(val_ds))

val_dl = DataLoader(val_ds, batch_size=B, shuffle=True)

Repo card metadata block was not found. Setting CardData to empty.


40000
4000
4000


In [3]:
src, tar = next(iter(val_dl))
src.shape

torch.Size([256, 130])

In [4]:
start = 10
end = 14
src_tensor = torch.stack(val_ds[start:end][0])
tar_tensor = torch.stack(val_ds[start:end][1])
seq_src = list(map(list , src_tensor))
seq_tar = list(map(list , tar_tensor))

In [5]:
to_list = lambda tensor: list(map(list , tensor))

In [6]:
tokenizer_src.decode_batch(seq_src), tokenizer_tar.decode_batch(seq_tar)

(['Hide process para _ meters',
  'solitaire ; cards ; klondike ; spider ; freecell ; patience ;',
  'Check Out Files',
  "If you ' re ever lost and alone in the woods , hug a tree"],
 ['छुपाएँ process अनुच्छेद मीटर्स',
  'सोलिटेयर ; पत्ते ; क्लोंडाइक ; मकड़ी ; FreeCell ; धैर्य ;',
  'जाँच बाहर फ़ाइल',
  'यदि आप कभी जंगल में अकेले गुम हो जाएँ , तो किसी पेड़ के आगोश में जाएँ'])

In [7]:
transformer = Transformer(tokenizer_src.get_vocab_size(), tokenizer_tar.get_vocab_size(), 
                          d_model, n_h, n_layers , d_ff, seq_len, drop_prob, 
                          tokenizer_src.token_to_id("<pad>"), tokenizer_tar.token_to_id("<pad>")).to(device)

In [8]:
transformer.load_state_dict(torch.load("ckpts/transformer_40000_0.0001_10"))

<All keys matched successfully>

In [9]:
op = transformer(src_tensor.to(device), tar_tensor.to(device))

In [10]:
transalation = torch.argmax(op, dim=-1)
transalation.shape

torch.Size([4, 130])

In [11]:
transalation = list(map(list , transalation))
tokenizer_src.decode_batch(seq_src), tokenizer_tar.decode_batch(seq_tar), tokenizer_tar.decode_batch(transalation, skip_special_tokens=False)

(['Hide process para _ meters',
  'solitaire ; cards ; klondike ; spider ; freecell ; patience ;',
  'Check Out Files',
  "If you ' re ever lost and alone in the woods , hug a tree"],
 ['छुपाएँ process अनुच्छेद मीटर्स',
  'सोलिटेयर ; पत्ते ; क्लोंडाइक ; मकड़ी ; FreeCell ; धैर्य ;',
  'जाँच बाहर फ़ाइल',
  'यदि आप कभी जंगल में अकेले गुम हो जाएँ , तो किसी पेड़ के आगोश में जाएँ'],
 ['<sos> छुपाएँ process अनुच्छेद मीटर्स <eos> checksum checksum checksum checksum <sos> <sos> छवि checksum checksum checksum <sos> <sos> checksum checksum checksum slot slot जरूरी अंजुता checksum छवि checksum <sos> slot <sos> slot slot slot checksum checksum cdrkit slot कुछ बिंदु बिंदु अंजुता <sos> छवि slot <sos> slot checksum checksum प्लगइन checksum checksum checksum मिटाएँ slot <sos> जरूरत slot <sos> <sos> <sos> slot जरूरी slot slot slot slot checksum सर्वर जरूरी slot slot slot जरूरत checksum checksum जरूरी जरूरी slot जरूरी checksum जरूरत जरूरत slot जरूरी कुछ जरूरी slot कुछ जरूरी slot जरूरत लिया अंजुता छवि slo

### inference

In [12]:
torch.ones((2,5))*-1e9

tensor([[-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]])

In [13]:
torch.softmax(torch.ones((2,5))*-1e9, dim=-1)

tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])

In [14]:
curr_ind = 4
start = 100
end = 108
src_tensor = torch.stack(val_ds[start:end][0])
tar_tensor = torch.stack(val_ds[start:end][1])

curr_src_tensor = torch.ones(src_tensor.shape, dtype=src_tensor.dtype)
curr_src_tensor.copy_(src_tensor)
# curr_src_tensor[:,curr_ind+1] = tokenizer_src.token_to_id("<eos>") 
# curr_src_tensor[:, curr_ind+2:] = tokenizer_src.token_to_id("<pad>")

tensor([[  2,  46,  32,  ...,   1,   1,   1],
        [  2,   0, 522,  ...,   1,   1,   1],
        [  2, 339,  11,  ...,   1,   1,   1],
        ...,
        [  2, 282,   8,  ...,   1,   1,   1],
        [  2,   7, 713,  ...,   1,   1,   1],
        [  2,  86,  42,  ...,   1,   1,   1]])

In [15]:
curr_tar_tensor = torch.ones(tar_tensor.shape, dtype=tar_tensor.dtype)
curr_tar_tensor.copy_(tar_tensor)
curr_tar_tensor[:, curr_ind:] = tokenizer_tar.token_to_id("<pad>")
curr_tar_tensor

tensor([[   2,  141,   67,  ...,    1,    1,    1],
        [   2,  843, 6862,  ...,    1,    1,    1],
        [   2, 1454,    3,  ...,    1,    1,    1],
        ...,
        [   2,   84,   96,  ...,    1,    1,    1],
        [   2,   73,   74,  ...,    1,    1,    1],
        [   2,   70,   36,  ...,    1,    1,    1]])

In [16]:
op = transformer(curr_src_tensor.to(device), curr_tar_tensor.to(device))

In [17]:
transalation = torch.argmax(op, dim=-1)
transalation.shape

torch.Size([8, 130])

In [18]:
transalation = list(map(list , transalation))
tokenizer_src.decode_batch(to_list(curr_src_tensor), skip_special_tokens=True), tokenizer_tar.decode_batch(to_list(tar_tensor), skip_special_tokens=True), tokenizer_tar.decode_batch(transalation, skip_special_tokens=False)

(['CD / DVD Creator Folder',
  'search',
  'Command :',
  'Shift',
  'Place the two of spades next to the ace of spades .',
  'jack of hearts',
  '_ Change Location',
  'Open an executable or attach to a process to start debugging .'],
 ['CD / DVD सर्जक फोल्डर',
  'अंजूटा हैल्प ब्राउजर',
  'कमांडः',
  'शिफ्ट',
  'हुकुम की दुक्का के बगल में हुकुम के इक्का को रखें .',
  'लाल पान का गुलाम',
  'बदलें स्थान ( _ C )',
  'खोलें या को a process को प्रारंभ नहीं'],
 ['<sos> CD / DVD slot slot slot slot slot slot slot checksum checksum slot slot slot checksum slot slot slot slot slot कुछ slot slot checksum सीडी checksum slot slot कुछ कुछ slot अंतिम slot सीडी सीडी कुछ सीवीएस सीडी कुछ सीडी सिर्फ कुछ सिर्फ जरूरी slot सिर्फ सीडी सीडी सिर्फ सीवीएस सीडी slot सिर्फ अंतिम सीडी कुछ <sos> अंतिम सीडी सीडी सीडी जरूरी slot slot slot कुछ सिर्फ slot slot slot slot slot slot slot slot slot slot slot slot जरूरत कुछ कुछ कुछ slot कुछ कुछ slot slot slot slot सीडी सिर्फ slot पर्याप्त सीडी slot कुछ छवि छवि छवि अंजुता 

In [65]:
start = 3500
end = 3508
src_tensor = torch.stack(val_ds[start:end][0])
tar_tensor = torch.stack(val_ds[start:end][1])

In [66]:
tar_tensor.shape

torch.Size([8, 130])

In [67]:
start = 2
curr_tar_tensor = torch.ones(tar_tensor.shape, dtype=tar_tensor.dtype)
curr_tar_tensor.copy_(tar_tensor)
curr_tar_tensor[:, 1:] = tokenizer_tar.token_to_id("<pad>")
curr_tar_tensor

tensor([[2, 1, 1,  ..., 1, 1, 1],
        [2, 1, 1,  ..., 1, 1, 1],
        [2, 1, 1,  ..., 1, 1, 1],
        ...,
        [2, 1, 1,  ..., 1, 1, 1],
        [2, 1, 1,  ..., 1, 1, 1],
        [2, 1, 1,  ..., 1, 1, 1]])

In [68]:
tar_sentences = tokenizer_tar.decode_batch(to_list(tar_tensor), skip_special_tokens=True)

In [69]:
def print_sents(arrs):
    n = len(arrs)
    for each in range(len(arrs[0])):
        print("*****************************************")
        for i in range(n):
            print(arrs[i][each])

In [70]:
op = transformer(src_tensor.to(device), curr_tar_tensor.to(device))
transalation = torch.argmax(op, dim=-1)
decoded_trans = tokenizer_tar.decode_batch(to_list(transalation[:, :curr_ind+1]), skip_special_tokens=False)
print_sents((tar_sentences,decoded_trans)) 

*****************************************
ढूंढा जा रहा है
<sos> कुछ checksum slot जरूरी checksum अंजुता slot कुछ आयात
*****************************************
स्लाइडर
<sos> स्लाइडर स्लाइडर checksum checksum अंतिम checksum checksum checksum <sos>
*****************************************
MD5 के लिए 0 , SHA1 के लिए 1 और SHA256 के लिए 2 सेट करें
<sos> अंतिम अंजुता आयात कुछ slot कुछ checksum छवि त्यागना
*****************************************
'% s ' फाइल में लिखने की त्रुटिः % s
<sos> कुछ छवि slot slot slot slot slot slot जरूरी
*****************************************
%. 1f × ( BD )%. 1f × ( DVD )%. 1f × ( CD )
<sos> कुछ छवि कुछ कुछ slot जरूरी जरूरी कुछ कुछ
*****************************************
पुष्टिकरण
<sos> मिटाएँ अंजुता मिटाएँ कुछ slot मान checksum छवि जरूरी
*****************************************
घटनाओं हेतु ध्वनि बजाएँ या नहीं .
<sos> slot कुछ slot slot त्यागना कुछ फंक्शन slot slot
*****************************************
तिथि तथा समय प्रविषश्ट करें
<sos> कुछ कुछ कुछ छवि f

In [ ]:
curr_tar_tensor[:,0] = tokenizer_tar.token_to_id("<sos>")
curr_tar_tensor[:, start:] = tokenizer_tar.token_to_id("<pad>")

In [29]:
start = 1
curr_tar_tensor = torch.ones(tar_tensor.shape, dtype=tar_tensor.dtype)
curr_tar_tensor.copy_(tar_tensor)
curr_tar_tensor[:,0] = tokenizer_tar.token_to_id("<sos>")
curr_tar_tensor[:, start:] = tokenizer_tar.token_to_id("<pad>")
for curr_ind in range(start,10):
    op = transformer(src_tensor.to(device), curr_tar_tensor.to(device))
    transalation = torch.argmax(op, dim=-1)
    
    decoded_trans = tokenizer_tar.decode_batch(to_list(transalation[:, :curr_ind+1]), skip_special_tokens=False)
    curr_tar_tensor[:, curr_ind] = transalation[:, curr_ind]
#     if decoded_trans[-1] == "<eos>":
#         break

print(decoded_trans)

['<sos> slot आयात slot checksum checksum slot छवि अंजुता आयात', '<sos> लिएः slot checksum अंतिम checksum छवि छवि छवि लिएः', '<sos> checksum slot कुछ कुछ डील कुछ अंतिम slot अंजुता', '<sos> checksum अंतिम कुछ कुछ slot कुछ slot छवि छवि', '<sos> लिएः मिटाएँ लिएः लिएः slot छवि लिएः डील डील', '<sos> कुछ रूप slot अंतिम अंतिम सिर्फ छवि छवि छवि', '<sos> कुछ कुछ slot checksum checksum जरूरी checksum जरूरी checksum', '<sos> slot कुछ कुछ कुछ कुछ checksum कुछ slot कुछ', '<sos> slot slot slot अभी slot slot slot कुछ जरूरी', '<sos> सेल्टिक सेल्टिक सेल्टिक सेल्टिक जरूरी सेल्टिक सेल्टिक चुनो सेल्टिक', '<sos> कुछ slot slot कुछ checksum checksum जरूरी slot कुछ', '<sos> checksum checksum अंतिम checksum अंतिम कुछ checksum checksum चुनो', '<sos> स्वतःपूर्ण slot आयात अंतिम सर्वर अंतिम checksum छवि slot', '<sos> छवि checksum checksum checksum slot checksum checksum checksum आयात', '<sos> आयात छवि कुछ slot कुछ slot slot checksum छवि', '<sos> कुछ कुछ मिटाएँ सामान्य कुछ अंतिम रिबेस डील सर्वर', '<sos> कुछ checksum

In [30]:
src_sentences = tokenizer_src.decode_batch(to_list(src_tensor), skip_special_tokens=True)
tar_sentences = tokenizer_tar.decode_batch(to_list(tar_tensor), skip_special_tokens=True)

for i in range(len(src_sentences)):
    print("*****************************************")
    print(src_sentences[i])
    print(tar_sentences[i])
    print(decoded_trans[i])

*****************************************
Open with
खोलें इसके साथ
<sos> slot आयात slot checksum checksum slot छवि अंजुता आयात
*****************************************
Simulation of image to disc burning
छवि की डिस्क बर्निंग का सिमुलेशन
<sos> लिएः slot checksum अंतिम checksum छवि छवि छवि लिएः
*****************************************
Diff files
डिफ फ़ाइल
<sos> checksum slot कुछ कुछ डील कुछ अंतिम slot अंजुता
*****************************************
That way , old files from previous sessions will be usable after burning .
उस तरह से , पिछले सत्र से पुरानी फाइलें लिखने के बाद प्रयोग लायक होगी .
<sos> checksum अंतिम कुछ कुछ slot कुछ slot छवि छवि
*****************************************
% s on reserve
एक खाली खाँचाstockslot hint
<sos> लिएः मिटाएँ लिएः लिएः slot छवि लिएः डील डील
*****************************************
Autocompletion
स्वतः समापन
<sos> कुछ रूप slot अंतिम अंतिम सिर्फ छवि छवि छवि
*****************************************
Project Manager
परियोजना प्रबंधक
<sos> कुछ कुछ slot c

In [161]:
transalation.shape

torch.Size([8, 130])

In [156]:
tokenizer_tar.decode_batch(to_list(transalation[:, :curr_ind+1]), skip_special_tokens=False)

['<sos> CD / DVD slot',
 '<sos> अंजूटा हैल्प ब्राउजर slot',
 '<sos> कमांडः <eos> लिएः अंतिम',
 '<sos> शिफ्ट <eos> checksum डील',
 '<sos> हुकुम की दुक्का आयात',
 '<sos> लाल पान का कुछ',
 '<sos> बदलें स्थान ( आयात',
 '<sos> खोलें या को checksum']

In [85]:

seq_src = list(map(list , src_tensor))
seq_tar = list(map(list , tar_tensor))

In [86]:
src_tensor

tensor([[   2,  467,  213, 3380,    7, 3365,    3,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 1332,  127,  280,  127, 4697,  127, 4801,  127, 4664,  127, 4736,
          127,    3,    1,    1,    1, 